In [1]:
import pygame
import heapq
import time

# Grid size
WIDTH, HEIGHT = 600, 600
ROWS, COLS = 20, 20
CELL_SIZE = WIDTH // COLS

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GREEN = (0, 255, 0)
RED = (255, 0, 0)
BLUE = (0, 0, 255)
YELLOW = (255, 255, 0)

# Initialize pygame
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("D* Pathfinding Visualization")

# Grid representation (0 = empty, 1 = obstacle)
grid = [[0 for _ in range(COLS)] for _ in range(ROWS)]
start, goal = None, None

# Directions for movement (right, left, down, up)
MOVES = [(0, 1), (0, -1), (1, 0), (-1, 0)]

def heuristic(a, b):
    """Calculates Manhattan distance between two points."""
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def draw_grid():
    """Draws the grid, including obstacles, start, and goal positions."""
    for y in range(ROWS):
        for x in range(COLS):
            color = WHITE
            if grid[y][x] == 1:
                color = BLACK  # Obstacles
            elif (y, x) == start:
                color = GREEN  # Start point
            elif (y, x) == goal:
                color = RED  # Goal point
            pygame.draw.rect(screen, color, (x * CELL_SIZE, y * CELL_SIZE, CELL_SIZE, CELL_SIZE))
            pygame.draw.rect(screen, BLACK, (x * CELL_SIZE, y * CELL_SIZE, CELL_SIZE, CELL_SIZE), 1)

def d_star(start, goal):
    """Implements a simplified D* pathfinding algorithm."""
    open_set = []  # Priority queue
    heapq.heappush(open_set, (0, start))
    came_from = {}
    cost = {start: 0}  # Cost from start to current node
    visited = set()
    
    while open_set:
        pygame.event.pump()
        _, current = heapq.heappop(open_set)
        visited.add(current)
        
        if current == goal:  # If we reached the goal, reconstruct path
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            return path[::-1]
        
        for move in MOVES:
            neighbor = (current[0] + move[0], current[1] + move[1])
            
            if 0 <= neighbor[0] < ROWS and 0 <= neighbor[1] < COLS and grid[neighbor[0]][neighbor[1]] == 0:
                temp_cost = cost[current] + 1
                
                if neighbor not in cost or temp_cost < cost[neighbor]:
                    came_from[neighbor] = current
                    cost[neighbor] = temp_cost
                    heapq.heappush(open_set, (temp_cost + heuristic(neighbor, goal), neighbor))
                    
                    # Visualizing the search process
                    pygame.draw.rect(screen, YELLOW, (neighbor[1] * CELL_SIZE, neighbor[0] * CELL_SIZE, CELL_SIZE, CELL_SIZE))
                    pygame.display.flip()
                    time.sleep(0.05)
    return None  # No path found

def main():
    """Main function to handle user input and visualization."""
    global start, goal
    running = True
    path = None
    
    while running:
        screen.fill(WHITE)
        draw_grid()
        
        # Draw the path if found
        if path:
            for p in path:
                pygame.draw.rect(screen, BLUE, (p[1] * CELL_SIZE, p[0] * CELL_SIZE, CELL_SIZE, CELL_SIZE))
        
        pygame.display.flip()
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            
            elif pygame.mouse.get_pressed()[0]:  # Left click to place obstacles
                x, y = pygame.mouse.get_pos()
                grid[y // CELL_SIZE][x // CELL_SIZE] = 1
            
            elif pygame.mouse.get_pressed()[2]:  # Right click to set start and goal
                x, y = pygame.mouse.get_pos()
                cell = (y // CELL_SIZE, x // CELL_SIZE)
                if not start:
                    start = cell
                elif not goal:
                    goal = cell
            
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE and start and goal:
                    path = d_star(start, goal)  # Start pathfinding
                elif event.key == pygame.K_c:  # Clear the grid
                    grid[:] = [[0 for _ in range(COLS)] for _ in range(ROWS)]
                    start, goal, path = None, None, None
    
    pygame.quit()

main()


pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html
